In [70]:
!pip install beautifulsoup4

In [1]:
# -*- coding: utf-8 -*-
sys.getdefaultencoding()

'utf-8'

In [5]:
# Funciona apenas com Python 3
# -*- coding: utf-8 -*-

# Próximas etapas: 
# - Testar nova modularização
# - Quebrar a lista de autores que estão com virgula
# - Incluir citação
# - Começar a baixar os PDFs

import requests
import time
import random
from bs4 import BeautifulSoup

last_url = ''
last_soup = ''
last_author = ''
PROXY_LIST = []

USER_AGENT_CHOICES = [
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:23.0) Gecko/20100101 Firefox/23.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.62 Safari/537.36',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.146 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.146 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64; rv:24.0) Gecko/20140205 Firefox/24.0 Iceweasel/24.3.0',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:28.0) Gecko/20100101 Firefox/28.0',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:28.0) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
]

from lxml.html import fromstring
import traceback

def get_proxies():
    url = 'https://free-proxy-list.net/'
    headers = {'user-agent': random.choice(USER_AGENT_CHOICES) }
    response = requests.get(url,  headers=headers)
    parser = fromstring(response.text)
    lista = []
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            lista.append(proxy)
    print('Proxies obtidos: %d' % len(lista))
    return lista

def get_page(url):
    global PROXY_LIST
    last_soup = ''
    headers = {'user-agent': random.choice(USER_AGENT_CHOICES) }
    while len(PROXY_LIST) > 0:
        try:
            proxy = random.choice(PROXY_LIST)
            response = requests.get(url, headers=headers, proxies={ "http": proxy, "https": proxy, },timeout=30)
            if response.status_code == 200:
                print('%s (%s)' % (url,proxy))
                if u'Ative o JavaScript' in response.text:
                    print('Proxy Blocked 1')
                    raise Exception('Blocked')
                last_soup = BeautifulSoup(response.text, 'html.parser')
                break
            else:
                print('Proxy Blocked 2')
                raise Exception('Blocked')
        except:
            PROXY_LIST.remove(proxy)
            print('Dead Proxy: %s - Left: %d' % (proxy, len(PROXY_LIST)) )
            while len(PROXY_LIST) == 0:
                time.sleep(60)
                PROXY_LIST = get_proxies()
            
    return last_soup

def find_artigos(soup):
    try:
        lista = soup.find('div', attrs={'id': 'gs_bdy'}).find('div', attrs={'id': 'gs_bdy_ccl'})
        lista = lista.find('div',attrs={'id': 'gs_res_ccl'})
        lista = lista.find('div',attrs={'id': 'gs_res_ccl_mid'})
    except:
        if soup:
            print('erro:'+soup.text)
        lista = None
    return lista

def parse_pagina(lista, artigos, origem):
    incluidos = 0
    for registro in lista.findAll('div',attrs={'class':'gs_r gs_or gs_scl'}):

        tipo = registro.find('div',attrs={'class':'gs_or_ggsm'})
        if tipo:
            tipo = tipo.find('span').text
        else:
            tipo = 'HTML'
            
        artigo = { 'tipo': tipo } # Contem a indicação se é HTML ou PDF
        
        item = registro.find('div',attrs={'class':'gs_ri'})
        divs = item.findAll('div')
        titulo = item.find('h3')
        if titulo and titulo.find('a'):
            artigo['url'] = titulo.find('a')['href']
            artigo['titulo'] =  titulo.find('a').text
        else:
            artigo['titulo'] = 'Titulo não encontrado (%d)' % incluidos
            artigo['url'] = last_url

        detalhe = divs[0].text.split('- ')
        autores = {}
        for autor in detalhe[0].split(','):
            nome = autor.strip()
            autores[ nome ] = ''
        
        for alink in divs[0].findAll('a'):
            nome = alink.text.split('- ')[0].strip()
            autores[ nome ] = alink['href'].split('&')[0]
    
        #for autor in sorted(autores.keys()):
        #    print('Autor :%s' % autor)
        #    print('Link  :%s' % autores[autor])
               
        artigo['autores'] = autores

        year = ''
        journal = ''
        if len(detalhe) > 1:
            journal = detalhe[1].strip()
            if len(journal.split(',')) > 1:
                year = journal.rsplit(',',1)[1].strip()
                journal = journal.split(',')[0].strip()
            else:
                if len(journal) == 4:
                    year = journal
                    journal = ''
                
        artigo['journal'] = journal
        artigo['year'] = year
        artigo['origem'] = origem

        try:
            item = divs[2]
            for celula in item.findAll('a'):
                if celula.text[0:3].upper() == 'CIT' and len(celula.text.split(' ')) > 1:   # Cited by ou citado por
                    artigo['citado'] = celula.text.split(' ')[2]
        except:
            artigo['citado'] = 0
        
        artigos.append( artigo )
        incluidos += 1
    return incluidos

PROXY_LIST = get_proxies()
if len(PROXY_LIST) == 0:
    raise Exception('No proxies found')

urls = [
    'https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=',
]       

# exemplo de url que cita um determinado autor
# https://scholar.google.com.br/scholar?hl=pt-BR&lr=lang_pt&as_sdt=0,5&q=label%3A+digital+humanities
# url2 = 'https://scholar.google.com.br/scholar?lr=lang_pt&hl=pt-BR&as_sdt=0,5&sciodt=0,5&cites=17950367634186489359,349589428917357430,1179427091885643356&scipsc='

url_index = 1
artigos = []
resultado_por_url = []
for url_base in urls:
    total = 0
    incluidos = 0 
    while True:
        last_url = url_base+'&as_vis=1&start=%d' % total
        soup = get_page( last_url )
        lista = find_artigos(soup)
        if lista:
            incluidos = parse_pagina(lista, artigos, url_index)
            total += incluidos
            if incluidos < 10:
                break
    time.sleep(10)
    url_index += 1
    resultado_por_url.append(total)
print('Fim')

Proxies obtidos: 10
Dead Proxy: 46.42.17.191:8080 - Left: 9
Dead Proxy: 103.12.160.200:53281 - Left: 8
Proxy Blocked 2
Dead Proxy: 177.18.195.76:8888 - Left: 7
https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=1&start=0 (188.115.129.41:53281)
Proxy Blocked 1
Dead Proxy: 188.115.129.41:53281 - Left: 6
Dead Proxy: 5.178.214.163:8080 - Left: 5
Proxy Blocked 2
Dead Proxy: 41.76.245.86:8080 - Left: 4
https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=1&start=0 (83.219.133.176:41258)
https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=1&start=10 (103.103.88.91:8080)
https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=1&start=20 (83.219.133.176:41258)
Dead Proxy: 93.103.12.195:53281 - Left: 3
https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=

https://scholar.google.com.br/scholar?hl=pt-BR&as_sdt=0%2C5&q=%22humanidades+digitais%22&btnG=&as_vis=1&start=520 (103.209.140.245:8080)
Fim


In [6]:
print(len(artigos))

524


In [7]:
numero = 0
for artigo in artigos:
    titulo = artigo['titulo']
    print(numero, titulo, artigo['autores'])
    numero += 1

0 Humanidades Digitais: Novos desafios e oportunidades {'D Guerreiro': '/citations?user=DV-nL-MAAAAJ', 'JL Borbinha': '/citations?user=4GFlVJQAAAAJ'}
1 Humanidades digitais: um campo praxiológico para mediações e políticas culturais? {'IPM Damian': '', 'MA ALMEIDA': ''}
2 Os Objetos Técnicos e seus papéis no horizonte das Humanidades Digitais: um caso para a Ciência da Informação. {'RM Pimenta': ''}
3 As humanidades digitais no cruzamento entre museus e turismo {'MI Roque': ''}
4 Academia, Geopolítica das Humanidades Digitais e Pensamento Crítico {'PS Pereira': ''}
5 Humanidades Digitais: novos desafios e Oportunidades (novo artigo) {'D Guerreiro': '/citations?user=DV-nL-MAAAAJ', 'J Borbinha': ''}
6 As Humanidades Digitais como uma comunidade de práticas dentro do formalismo académico: dos exemplos internacionais ao caso português {'D Alves': '/citations?user=EskYPIcAAAAJ'}
7 O Corpus Tycho Brahe: contribuições para as humanidades digitais no Brasil {'MCP de Sousa': '/citations?user=jW

In [8]:
import csv
with open('hd.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['titulo','autores','ano','citacoes','tipo','url','origem'])
    for artigo in artigos:
        if 'citado' in artigo:
            citacoes = artigo['citado']
        else:
            citacoes = 0
        writer.writerow([artigo['titulo'],artigo['autores'],artigo['year'],citacoes,artigo['tipo'],artigo['url'],artigo['origem']])
print('arquivo gravado com sucesso!')

arquivo gravado com sucesso!


In [9]:
base = set()
dedup = []
for d in artigos:
    t = tuple(d['titulo'].strip())
    if t not in base:
        base.add(t)
        dedup.append(d)

print(len(dedup))

import csv
with open('dedup.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['titulo','autores','ano','citacoes','tipo','url','origem'])
    for artigo in dedup:
        if 'citado' in artigo:
            citacoes = artigo['citado']
        else:
            citacoes = 0
        writer.writerow([artigo['titulo'],artigo['autores'],artigo['year'],citacoes,artigo['tipo'],artigo['url'],artigo['origem']])
print('arquivo gravado com sucesso!')

517
arquivo gravado com sucesso!


In [169]:
from collections import Counter

totais = Counter()
autores = {}
for reg in dedup:
    for autor, link in reg['autores'].items():
        autor_normalizado = autor.replace('…','').strip()
        if autor_normalizado in autores:
            if autores[autor_normalizado] == '' and link != '':
                autores[autor_normalizado] = link
            if autores[autor_normalizado] != link:
                print('different user: %s %s %s' % (autor_normalizado,autores[autor_normalizado],link))
            totais[autor_normalizado] += 1
        else:
            autores[autor_normalizado] = link
            
print(len(autores))
for top in totais.most_common(10):
    print(top)

different user: MP Kreitlon /citations?user=sA2OEWYAAAAJ 
different user: JA Valente /citations?user=5ZSxmZUAAAAJ 
different user: P Nogueira /citations?user=8bLk3poAAAAJ /citations?user=aW6zi9cAAAAJ
different user: R Torres /citations?user=qfmF53UAAAAJ 
different user: J Revez /citations?user=BIrnGbMAAAAJ 
different user: LG Furini /citations?user=1teP_icAAAAJ 
different user: I Carvalho /citations?user=zzT5VboAAAAJ /citations?user=9AdbZ-cAAAAJ
different user: I Carvalho /citations?user=zzT5VboAAAAJ /citations?user=9AdbZ-cAAAAJ
1587
('S Barbosa', 10)
('M Portela', 7)
('MEK Buzato', 6)
('RC Paraizo', 6)
('LC Petry', 6)
('E Felinto', 6)
('CP Júnior', 5)
('A Digitais', 5)
('H Elias', 5)
('R de Lima Soares', 4)
